Import required libraries

In [ ]:
from sys import argv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks")

Import and concatenate data for input visualization

In [ ]:
Step = pd.read_csv('InduciblePromoter_Step.csv', delimiter=",")
Pulse = pd.read_csv('InduciblePromoter_Pulse.csv', delimiter=",")
Random = pd.read_csv('InduciblePromoter_Random.csv', delimiter=",")
Optimised = pd.read_csv('InduciblePromoter_OptimisedValues_eSS.csv', delimiter=",")

Data = pd.concat([Step, Pulse, Random, Optimised])
print(Data)

Extract information relative to the parameter fit after 50 hours of experiment

In [ ]:
SubData_50 = Data.query('hour == 50')
SubData_50_type = SubData_50.query('type == "PARAM_FIT"')

Add to the dataframe a column with the relative error

In [ ]:
d = SubData_50_type.set_index('runName') # set the name of the run as index to the dataframe
d['abslog2Ratio'] = np.absolute(np.log2(d['value']/d['trueValue']))   # add a column to the dataframe with the absolute value of the log2-ratio between estimate and true parameter value, for each experiment
Data_50_type_update = d.reset_index()


In [ ]:
print(Data_50_type_update)

Import the data required for panel a) Example of optimised input trace, pseudo-data and response of the fitted model to this data

In [ ]:
# Load pseudo experimental data
PseudoData_Opt = pd.read_csv('PseudoExperimentalData_OpteSS.csv', delimiter=",")

# Load input trace
Input_Opt = pd.read_csv('Input_OpteSS.csv', delimiter=",")

# Load simulated output
SimPseudo_Opt = pd.read_csv('SimulationPseudo_OpteSS.csv', delimiter=",")

Code for figure 4

In [ ]:
import matplotlib.gridspec as gridspec
fig = plt.figure(figsize=(6.8, 5.8), dpi = 600)
sns.set_context("paper", rc={"font.size":8,"axes.titlesize":8,"axes.labelsize":8,"axes.tickslabelssize":8,"text.usetex" : True,"text.latex.unicode" : True})  
pal = dict(alpha1='#fff7f3', Vm1 = '#fde0dd',h1 = '#fcc5c0', Km1 = '#fa9fb5', d1 ='#f768a1' , alpha2 = '#dd3497', d2 ='#ae017e' ,Kf='#7a0177' )

gs0 = gridspec.GridSpec(2, 1)
gs0.update(left=0.05, right=0.35)
ax0 = plt.subplot(gs0[0, :])


#------> Plot of prototype of optimised input(ax1), experimental data and output (ax0)
ax0.plot(np.array(SimPseudo_Opt['Var1']),np.array(SimPseudo_Opt['Citrine_molec_opt']),color =(0/255, 166/255, 80/255),linewidth = 1.2)
# plot Pseudodata
ax0.scatter(np.array(PseudoData_Opt['Var1']),np.array(PseudoData_Opt['Citrine_molec_opt']),marker = '.',color=(0/255, 166/255, 80/255),s =15)#, alpha = 0.5)
ax0.tick_params(length=2)
#ax0.set_ylabel('Citrine (molecules)', color = (0/255, 166/255, 80/255))
ax0.tick_params('y', colors=(0/255, 166/255, 80/255))
ax0.set_yticks([200,400,600,800,1000,1200])
ax0.set_yticklabels(('0.2','0.4','0.6','0.8','1','1.2'))
ax0.set_ylabel('Citrine ($10^3$ molecules)',color = (0/255, 166/255, 80/255))
ax0.set_xlabel('Time (hours)')

#---
ax1 = ax0.twinx()
ax1.plot(np.array(Input_Opt['Var1']),np.array(Input_Opt['Var2']),color =(237/255, 28/255, 36/255),linewidth = 1)
#ax1.set_ylabel('IPTG ($\mu$M)', color=(237/255, 28/255, 36/255))
ax1.tick_params('y', colors=(237/255, 28/255, 36/255))
ax1.tick_params(length=2)

ax1.spines['top'].set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.spines['left'].set_visible(False)
ax1.set_yticks([0,200,400,600,800,1000])
ax1.set_yticklabels(('0','0.2','0.4','0.6','0.8','1'))
ax1.set_ylabel('IPTG ($10^3 \mu$M)',color = (237/255, 28/255, 36/255),rotation = -90)

#------> Bar plot of average relative error 
pal1 = dict(S='#e6f5c9', P = '#fff2ae',R = '#bebada', O = '#fcbba1')
#dict(S='#5e3c99', P = '#b2abd2',R = '#fdb863', O = '#fcbba1')#8dd3c7
#ffffb3
#bebada
ax2 = plt.subplot(gs0[1, :])
ax2 = plt.axhline(y=0,c = 'black',linewidth = 0.5)
ax2 = sns.barplot(data = Data_50_type_update,x = "exptName",y = "abslog2Ratio",order =['S','P','R','O'], palette = pal1,edgecolor = 'black',errwidth = 0.8,alpha=.8)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.tick_params(length=2)
ax2.set_xticklabels(('Step','Pulse','Random','Optimised'),rotation = -10)
ax2.set_xlabel('Input class')
ax2.set_ylabel(r'$\bar{\varepsilon}$ (-)',rotation = 0)

#------> Box+Swarm plot of relative error for each input class and parameter
gs1 = gridspec.GridSpec(4, 1)
gs1.update(left=0.55, hspace=0.1)#0.05

ax3 = plt.subplot(gs1[0, :])
ax3 = plt.axhline(y=0,c='b',linewidth=1)
ax3 = sns.swarmplot(Data_50_type_update.loc[Data_50_type_update['exptName'] == "S",'param'],Data_50_type_update.loc[Data_50_type_update['exptName'] == "S",'abslog2Ratio'],order = ['alpha1','Vm1','h1','Km1','d1','alpha2','d2','Kf'],size=2, color="k", linewidth=0,alpha = .2)
ax3 = sns.boxplot(Data_50_type_update.loc[Data_50_type_update['exptName'] == "S",'param'],Data_50_type_update.loc[Data_50_type_update['exptName'] == "S",'abslog2Ratio'],palette = pal,fliersize = 0,linewidth=0.5)
ax3.spines['right'].set_visible(False)
ax3.spines['top'].set_visible(False)
ax3.tick_params(length=2)
ax3.set_ylabel('')
ax3.set_title('Step',y = 0.85)

ax4 = plt.subplot(gs1[1, :])
ax4 = plt.axhline(y=0,c='b',linewidth=1)
ax4 = sns.swarmplot(Data_50_type_update.loc[Data_50_type_update['exptName'] == "P",'param'],Data_50_type_update.loc[Data_50_type_update['exptName'] == "P",'abslog2Ratio'],order = ['alpha1','Vm1','h1','Km1','d1','alpha2','d2','Kf'],size=2, color="k", linewidth=0,alpha = .2)
ax4 = sns.boxplot(Data_50_type_update.loc[Data_50_type_update['exptName'] == "P",'param'],Data_50_type_update.loc[Data_50_type_update['exptName'] == "P",'abslog2Ratio'],palette = pal,fliersize = 0,linewidth=0.5)
ax4.spines['right'].set_visible(False)
ax4.spines['top'].set_visible(False)
ax4.tick_params(length=2)
ax4.set_ylabel('')
ax4.set_title('Pulse',y = 0.85)

ax4.text(-2.5, -0.5, r'$\varepsilon$(-)')


ax5 = plt.subplot(gs1[2, :])
ax5 = plt.axhline(y=0,c='b',linewidth=1)
ax5 = sns.swarmplot(Data_50_type_update.loc[Data_50_type_update['exptName'] == "R",'param'],Data_50_type_update.loc[Data_50_type_update['exptName'] == "R",'abslog2Ratio'],order = ['alpha1','Vm1','h1','Km1','d1','alpha2','d2','Kf'],size=2, color="k", linewidth=0,alpha = .2)
ax5 = sns.boxplot(Data_50_type_update.loc[Data_50_type_update['exptName'] == "R",'param'],Data_50_type_update.loc[Data_50_type_update['exptName'] == "R",'abslog2Ratio'],palette = pal,fliersize = 0,linewidth=0.5)
ax5.spines['right'].set_visible(False)
ax5.spines['top'].set_visible(False)
ax5.tick_params(length=2)
ax5.set_ylabel('')
ax5.set_title('Random',y = 0.85)


ax6 = plt.subplot(gs1[3, :])
ax6 = plt.axhline(y=0,c='b',linewidth=1)
ax6 = sns.swarmplot(Data_50_type_update.loc[Data_50_type_update['exptName'] == "O",'param'],Data_50_type_update.loc[Data_50_type_update['exptName'] == "O",'abslog2Ratio'],order = ['alpha1','Vm1','h1','Km1','d1','alpha2','d2','Kf'],size=2, color="k", linewidth=0,alpha = .2)
ax6 = sns.boxplot(Data_50_type_update.loc[Data_50_type_update['exptName'] == "O",'param'],Data_50_type_update.loc[Data_50_type_update['exptName'] == "O",'abslog2Ratio'],palette = pal,fliersize = 0,linewidth=0.5)
ax6.spines['right'].set_visible(False)
ax6.spines['top'].set_visible(False)
ax6.tick_params(length=2)
ax6.set_xticklabels([r'$\alpha$',r'$v$',r'$h$',r'$K_{r}$',r'$\gamma$',r'$k_{p}$',r'$\gamma_{f}$',r'$k_{f}$'])
ax6.set_xlabel('Parameter')
ax6.set_ylabel('')
ax6.set_title('Optimised',y = 0.85)



for ax in [ax3, ax4,ax5]:
    plt.setp(ax.get_xticklabels(), visible=False)
    ax.set_xlabel('')



plt.show()
